In [1]:
import numpy as np

from monte_carlo_construction import monte_carlo
from RCA import RCA
from iGA import Improved_GA

import sys
import os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))
from capacity_estimation.calc_capacity import make_BMI_list 

In [28]:


class constant_class():
    const=RCA()
    N=1024
    K=512
    
    #print(BICM_int)
    design_SNR=0.0
    M=16

cst=constant_class()

def construction(cst,BICM_int):
    
    frozen_bits,info_bits=cst.const.main_const(cst.N,cst.K,cst.design_SNR,cst.M,BICM_int=BICM_int)

    BICM_int=np.concatenate([info_bits,frozen_bits])

    tmp=make_BMI_list(cst.design_SNR,cst.M)

    argtmp=np.argsort(tmp[:len(tmp)//2])

    BICM_int=np.reshape(BICM_int,[int(np.log2(cst.M**(1/2))),-1],order='C')

    BICM_int=BICM_int[argtmp,:]

    BICM_int=np.ravel(BICM_int,order='F')
    
    interleaver_check(cst,BICM_int,info_bits)
    
    return BICM_int

In [29]:
#確認
def interleaver_check(cst,BICM_int,info_bits):
    
    BICM_deint=np.argsort(BICM_int)
    
    tmp=make_BMI_list(cst.design_SNR,cst.M)
    for a in range(len(tmp)):
        tmp[a]=cst.const.calc_J_inv(tmp[a])
    #print(tmp)
    gamma=np.tile(tmp,cst.N//int(np.log2(cst.M)))

    xi=np.log(gamma)
    xi=xi[BICM_deint]

    #print(np.sort(np.argsort(xi)[:len(xi)//2]))
    #print(info_bits)

    if np.all(np.sort(np.argsort(xi)[:len(xi)//2])==info_bits)==False:
        print("interleaver error!!")

In [36]:
count=0
BICM_int=np.arange(cst.N)
#BICM_int_new=np.arange(cst.N)
while True:
    count+=1
    print("count:",count)
    BICM_int_new=construction(cst,BICM_int)
    if np.all(BICM_int_new==BICM_int)==True:
        break
    else:
        BICM_int=BICM_int_new

print(BICM_int)    
    

count: 1
count: 2
count: 3
count: 4
count: 5
count: 6
[   0  126    1 ... 1022  896 1023]
